# Clustering

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

## Description

A large technology firm has been hacked. Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.

The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacking').getOrCreate()

In [3]:
dataset = spark.read.csv("data/hack_data.csv", header=True, inferSchema=True)

In [5]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

## Create a feature vector

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
final_data = VectorAssembler(inputCols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used','Servers_Corrupted', 
                                          'Pages_Corrupted','WPM_Typing_Speed'], 
                             outputCol='features').transform(dataset).select("features")

In [10]:
# Apply scaling to the features
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)
# Normalize each feature to have unit standard deviation.
data = scalerModel.transform(final_data)

In [15]:
data.select('scaledFeatures').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------+
|scaledFeatures                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------+
|[0.5678510846650524,1.3658432518957642,1.9975768336483841,1.2858903881191532,2.2849485348398866,5.396290958577967] |
|[1.419627711662631,2.517986463945197,0.0,1.320644182392644,2.9377909733655687,5.150971112595909]                   |
|[2.2004229530770782,1.2444124562517545,1.9975768336483841,1.611707209433128,2.6113697541027276,5.262819066691072]  |
|[0.1419627711662631,0.7965469045293562,1.9975768336483841,1.0773676224782096,2.6113697541027276,5.279223433291696] |
|[1.419627711662631,1.4266459597520256,0.0,1.5508880694545193,2.6113697541027276,5.315014778602148]                 |
|[0.07098138558313155,1.3644462913476594,1.9975768336483

## Perform clustering

In [16]:
from pyspark.ml.clustering import KMeans

In [17]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [18]:
model_k3 = kmeans3.fit(data)
model_k2 = kmeans2.fit(data)

In [19]:
wssse_k3 = model_k3.computeCost(data)
wssse_k2 = model_k2.computeCost(data)

In [20]:
print("With K=3")
print("Within Set Sum of Squared Errors = " + str(wssse_k3))
print('--'*30)
print("With K=2")
print("Within Set Sum of Squared Errors = " + str(wssse_k2))

With K=3
Within Set Sum of Squared Errors = 434.1492898715845
------------------------------------------------------------
With K=2
Within Set Sum of Squared Errors = 601.7707512676716


WSSSE doesn't inform of us much about our question. This is becasue, we would expect that as K increases, the WSSSE decreases. 

We could however continue the analysis by seeing the drop from K=3 to K=4 to check if the clustering favors even or odd numbers. 

This won't be substantial, but its worth a look:

In [21]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(data)
    wssse = model.computeCost(data)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
Within Set Sum of Squared Errors = 601.7707512676716
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 434.1492898715845
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 419.2726748855374
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 251.65121348945024
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 228.3451465811906
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 235.97443081537682
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 202.89517002262062
------------------------------------------------------------


**Lets use the HINT provided my the forensic expert**

The attacks should be evenly numbered between the hackers

In [22]:
model_k3.transform(data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [23]:
model_k2.transform(data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



**The data is equally devided for 2 clusters.. So the attck was conducted by 2 hackers**